<div dir='rtl'>
<h1>پروژه اول</h1>
</div>

In [3]:
from timeit import default_timer as timer
from copy import deepcopy, copy
from collections import deque
import heapq

In [4]:
INPUT_COUNT = 3
INPUT_PATH  = 'data/input%d.txt'
TIMER_TEST_COUNT = 3

In [39]:
class Graph:
    def __init__(self, n, m):
        self.n = n
        self.m = m
        self.current = 0
        self.parent = None
        self.adj = [set() for _ in range(n)]
        self.difficulties = {}
        self.requirements = {}
        self.remainingTime = 0

    def __deepcopy__(self, memo):
        g = Graph(self.n, self.m)
        g.current = self.current
        g.parent = self.parent
        g.adj = copy(self.adj)
        g.difficulties = deepcopy(self.difficulties)
        g.requirements = deepcopy(self.requirements)
        g.remainingTime = self.remainingTime
        return g

    def __key(self):
        return (self.current, self.remainingTime, self.difficulties, self.requirements, self.parent.current if self.parent else None)

    def __str__(self):
        return str(self.__key())

In [40]:
def readInput(path):
    with open(path, 'r', encoding='utf-8') as f:
        n, m = map(int, f.readline().split())
        graph = Graph(n, m)
        for _ in range(m):
            u, v = map(int, f.readline().split())
            graph.adj[u - 1].add(v - 1)
            graph.adj[v - 1].add(u - 1)
        h = int(f.readline())
        for _ in range(h):
            u = int(f.readline()) - 1
            graph.difficulties[u] = -1
        s = int(f.readline())
        for _ in range(s):
            inp = f.readline().split()
            p = int(inp[0]) - 1
            vertices = {int(x) - 1 for x in inp[2:]}
            graph.requirements[p] = vertices
        i = int(f.readline()) - 1
        graph.current = i
    return graph

In [7]:
g1, g2, g3 = [readInput(INPUT_PATH % i) for i in range(1, INPUT_COUNT + 1)]

In [8]:
def searchTime(g, method):
    sum = 0
    for _ in range(TIMER_TEST_COUNT):
        gCopy = deepcopy(g)
        start = timer()
        method(gCopy)
        end = timer()
        sum += end - start
    return sum / TIMER_TEST_COUNT

In [9]:
def moveInGraph(g, dest):
    if dest not in g.adj[g.current]:
        return None
    newGraph = deepcopy(g)
    newGraph.current = dest
    newGraph.parent = g
    if dest in newGraph.difficulties:
        newGraph.difficulties[dest] += 1
        newGraph.remainingTime = newGraph.difficulties[dest]
    for _, vertices in newGraph.requirements.items():
        if dest in vertices:
            vertices.remove(dest)
    if dest in newGraph.requirements and len(newGraph.requirements[dest]) == 0:
        newGraph.requirements.pop(dest)
    return newGraph


def isGoal(g):
    return len(g.requirements) == 0


def getPathAndCost(g):
    path = []
    while g is not None:
        path.append(g.current + 1)
        g = g.parent
    path.reverse()
    additions = [0 if diff == -1 else diff for diff in g1.difficulties.values()]
    return path, len(path) + sum(additions) - 1

In [41]:
def bfs(g):
    discovered = set()
    queue = deque([g])
    discovered.add(str(g))
    while queue:
        g = queue.popleft()
        if isGoal(g):
            return getPathAndCost(g)
        if g.remainingTime > 0:
            g.remainingTime -= 1
            queue.append(g)
            continue
        for dest in g.adj[g.current]:
            newState = moveInGraph(g, dest)
            if newState is not None and str(newState) not in discovered:
                queue.append(newState)
                discovered.add(str(newState))
    return None, None

In [42]:
for i, g in enumerate([g1, g2, g3]):
    print(f'BFS: Input {i + 1}')
    path, cost = bfs(g)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
    else:
        print('No solution')
    print()

BFS: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8

BFS: Input 2
Path: 28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 22 -> 28
Cost: 12

BFS: Input 3
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21



In [43]:
for i, g in enumerate([g1, g2, g3]):
    print(f'BFS: Input {i + 1}')
    print(f'Time: {searchTime(g, bfs):.4f} seconds')
    print()

BFS: Input 1
Time: 0.0067 seconds

BFS: Input 2
Time: 7.6076 seconds

BFS: Input 3
Time: 4.1647 seconds

